# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [75]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [52]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     603 non-null    int64  
 1   City        602 non-null    object 
 2   Lat         603 non-null    float64
 3   Lng         603 non-null    float64
 4   Max Temp    603 non-null    float64
 5   Humidity    603 non-null    int64  
 6   Cloudiness  603 non-null    int64  
 7   Wind Speed  603 non-null    float64
 8   Country     161 non-null    object 
 9   Date        603 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 47.2+ KB


In [28]:
# Check data to be plotted for null values
city_data_df.loc[city_data_df["Humidity"].isnull()]

# Check coordinates for out-of-range values
city_data_df.loc[(city_data_df["Lng"] > 180) |
    (city_data_df["Lng"] < -180) |
    (city_data_df["Lat"] < -90) |
    (city_data_df["Lat"] > 90)
]

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [137]:
# Configure and display the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="Max Temp",
    cmap="rainbow",
    width = 800,
    height = 600
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [133]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5) &
    (city_data_df["Cloudiness"] < 100) &
    (city_data_df["Humidity"] > 0) &
    (city_data_df["Max Temp"] > 0) &
    (city_data_df["Max Temp"] < 45) &
    (city_data_df["Country"].isin(["US", "CA", "UK", "AU", "NZ"]))
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,toga village,-20.9792,124.9445,34.88,14,2,1.81,AU,1730386800
57,57,fortuna,-31.8088,148.8146,15.81,56,68,1.64,AU,1730386800
117,117,la passe,-43.2597,172.6143,10.64,74,95,3.46,NZ,1730386800
153,153,flying fish cove,-34.7692,142.1004,14.86,50,20,4.49,AU,1730386800
232,232,dunoon,36.2017,-82.3438,19.67,78,36,2.01,US,1730386800
233,233,nsanje,47.0337,-108.5449,2.10,49,31,3.90,US,1730386800
241,241,mogocha,41.9697,-104.4712,4.70,50,14,3.87,US,1730386800
356,356,kimbe,45.4591,-69.5349,17.43,87,84,2.63,US,1730386800
401,401,howard springs,38.9216,-101.1734,6.56,76,1,1.60,US,1730386800
449,449,valparaiso,58.5928,-161.6548,3.42,86,71,1.09,US,1730386800


### Step 3: Create a new DataFrame called `hotel_df`.

In [134]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,toga village,AU,-20.9792,124.9445,14,
57,fortuna,AU,-31.8088,148.8146,56,
117,la passe,NZ,-43.2597,172.6143,74,
153,flying fish cove,AU,-34.7692,142.1004,50,
232,dunoon,US,36.2017,-82.3438,78,
233,nsanje,US,47.0337,-108.5449,49,
241,mogocha,US,41.9697,-104.4712,50,
356,kimbe,US,45.4591,-69.5349,87,
401,howard springs,US,38.9216,-101.1734,76,
449,valparaiso,US,58.5928,-161.6548,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [107]:
import json
base_url = "https://api.geoapify.com/v2/places"
name_address = requests.get(f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:148.8146,-31.8088,10000&bias=proximity:148.8146,-31.8088&limit=20&apiKey=d309449bc1c44c3c98b523b0c28423de")
# print(json.dumps(name_address, indent=4, sort_keys=True))
print(name_address.status_code)

200


In [136]:
import time
# Set parameters to search for a hotel
radius = 10000 # Search radius in meters
params = {
    "categories": "accommodation.hotel", # Filter for hotel type accommodations
    "apiKey": geoapify_key,
    "limit": 1 # One search result only
}

apiKey = geoapify_key
categories = "accommodation.hotel"
limit = 1

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    filter = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # name_address = requests.get(base_url, params = params)
    name_address = requests.get(f"https://api.geoapify.com/v2/places?categories=accommodation&filter=circle:{lng},{lat},10000&bias=proximity:{lng},{lat}&limit=20&apiKey=d309449bc1c44c3c98b523b0c28423de")
    print(name_address.status_code)

    # Convert the API response to JSON format (JSON-ify)
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    time.sleep(1)
# Display sample data
hotel_df

Starting hotel search
200
toga village - nearest hotel: No hotel found
200
fortuna - nearest hotel: No hotel found
200
la passe - nearest hotel: Ashley Hotel
200
flying fish cove - nearest hotel: No hotel found
200
dunoon - nearest hotel: Super 8
200
nsanje - nearest hotel: No hotel found
200
mogocha - nearest hotel: No hotel found
200
kimbe - nearest hotel: Kino View
200
howard springs - nearest hotel: No hotel found
200
valparaiso - nearest hotel: No hotel found
200
jiangyin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,toga village,AU,-20.9792,124.9445,14,No hotel found
57,fortuna,AU,-31.8088,148.8146,56,No hotel found
117,la passe,NZ,-43.2597,172.6143,74,Ashley Hotel
153,flying fish cove,AU,-34.7692,142.1004,50,No hotel found
232,dunoon,US,36.2017,-82.3438,78,Super 8
233,nsanje,US,47.0337,-108.5449,49,No hotel found
241,mogocha,US,41.9697,-104.4712,50,No hotel found
356,kimbe,US,45.4591,-69.5349,87,Kino View
401,howard springs,US,38.9216,-101.1734,76,No hotel found
449,valparaiso,US,58.5928,-161.6548,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [144]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="Country",
    cmap="rainbow",
    width = 800,
    height = 600,
    hover_cols=["City", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City,Hotel Name)